In [17]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import sys
import threading
import csv

In [18]:
tsp_dir = os.getcwd() + os.sep + 'TSPLIB' + os.sep

In [19]:
tsp_files_list = os.listdir(tsp_dir)

In [20]:
def getOptmSolns():
    file = open(tsp_dir + os.sep + 'solutions', 'r')
    lines = file.readlines()
    resDict = {}
    for l in lines:
        s = l.split(' : ')
        s[0] += '.tsp'
        s[1] = s[1][:-1]
        resDict[s[0]] = float(s[1])
    return resDict

In [21]:
optmSolns = getOptmSolns()

In [22]:
subprocess.run('make christofides', shell=True, capture_output=True)

CompletedProcess(args='make christofides', returncode=0, stdout=b"make: 'christofides' is up to date.\n", stderr=b'')

In [23]:
def parseOutputString(outputStr):
    lines = outputStr.split('\n')
    resultDict = {}
    for i in range(len(lines)-1):
        line = lines[i]
        s = line.split()
        if(i == 0):
            resultDict[i] = [s[1]]
        else:
            resultDict[i] = s[1:]
    return resultDict

In [24]:
def getSupportedFilesList():
    tsp_files_list = os.listdir(tsp_dir)
    supportedFiles = []
    keyWord = 'EDGE_WEIGHT_SECTION'
    for filename in tsp_files_list:
        if(filename.find('.tsp') == -1):
            continue
        foundKeyWord = False
        file = open(tsp_dir + os.sep + filename, 'r')
        lines = file.readlines()
        for line in lines:
            if(line.find(keyWord) != -1):
                foundKeyWord = True
                break
        if(foundKeyWord == False):
            supportedFiles.append(filename)
    return supportedFiles

In [25]:
supported_tsp_files_list = getSupportedFilesList()

In [26]:
def launchChristofides(filename = 'gr48.tsp', perturbation_percentage = 0.0):
    filename = filename
    if(filename not in supported_tsp_files_list):
        print('WARNING: You are running the algorithm on an unsupported file, Perturbation does not work.')
    filename = 'TSPLIB' + os.sep + filename
    command = './christofides' + ' -f ' + filename + ' -p ' + str(perturbation_percentage)
    out = subprocess.run([command], shell=True, capture_output=True)
    output = out.stdout.decode('utf-8')
    resultDict = parseOutputString(output)
    return resultDict

In [27]:
def launchChristofidesThreadSafe(resultDictList, listLock, filename = 'gr48', perturbation_percentage = 0.0):
    dictToAdd = launchChristofides(filename, perturbation_percentage)
    listLock.acquire()
    resultDictList.append(dictToAdd)
    listLock.release()
    return    

In [28]:
def runExpt(filename, perturbation_percentage, num_threads = 15):
    resultList = []
    listLock = threading.Lock()
    threads = []
    for i in range(num_threads):
        t = threading.Thread(target = launchChristofidesThreadSafe, args = (resultList, listLock, filename, perturbation_percentage))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    return resultList

In [29]:
def getAvgList(inpDict=[]):
    l = [0.0 for i in range(5)]
    n = len(inpDict)
    for d in inpDict:
        try:
            l[0] += float(d[0][0])
        except:
            n -= 1
            continue
        for i in range(4):
            l[i+1] += float(d[i+1][1])
    if(n == 0):
        print('fhere', end ='\t')
        return l
    for i in range(5):
        l[i] = l[i]/n
    return l

In [30]:
def getNumFromString(s):
    numStr = ''
    for c in list(s):
        if(c.isdigit()):
            numStr += (c)
    return int(numStr)

In [31]:
def getData(MAX_N = 1000, NUM_OBSERVATIONS = 15, NUM_FILES = len(supported_tsp_files_list), ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]):
    data = {}
    if(NUM_FILES > len(supported_tsp_files_list)):
        print('There are only', len(supported_tsp_files_list), 'files available')
        NUM_FILES = len(supported_tsp_files_list)
    for i in range(NUM_FILES):
        filename = supported_tsp_files_list[i]
        if(getNumFromString(filename) > MAX_N):
            continue
        print(filename, end='\t')
        for j in range(len(ppList)):
            pp = ppList[j]
            output = runExpt(filename, pp, NUM_OBSERVATIONS)
            print(pp, end = '\t')
            l = getAvgList(output)
            data[(filename, pp)] = []
            for k in range(5):
                data[(filename, pp)].append(l[k])
        print(u'\u2713')
    return data

In [32]:
ppList = [0.0, 0.01, 0.02, 0.1, 0.3, 1, 2, 3, 5, 10]

In [ ]:
##### Uncomment last to run for different parameters, saved variable data is for MAX_N = 1000, NUM_OBS = 20, NUM_FILES = 100
# %store -r data
data = getData(MAX_N = 1000, NUM_OBSERVATIONS = 20, NUM_FILES = 100, ppList=ppList)

There are only 93 files available
a280.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
ali535.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
att48.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
att532.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
berlin52.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
bier127.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
burma14.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
ch130.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
ch150.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
d198.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
d657.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
dsj1000.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
eil101.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
eil51.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
eil76.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
fl417.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
gil262.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
gr137.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
gr202.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
gr229.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
gr431.tsp	0.0	0.01	0.02	0.1	0.3	1	2	3	5	10	✓
gr666.tsp	0

In [ ]:
# print(data)

In [ ]:
# ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
# ppList = [0.0, 0.01, 0.02, 0.1, 0.3, 1, 2, 3, 5, 10]
allTables = []
for t in range(5):
    table = []
    l1 = []
    l1.append('FileName\Heur: ' + str(t))
    for pp in ppList:
        l1.append(pp)
    table.append(l1)
    for file in supported_tsp_files_list:
        if((file, 0.0) not in data.keys()):
            continue
        l = []
        l.append(file)
        reqdStr = ''
        for pp in ppList:
            num = data[(file, pp)][t]
            optNum = optmSolns[file]
            reqdStr = str(round((num/optNum), 3)) + ' | ' + str(round(num, 3))
            l.append(reqdStr)
        table.append(l)
    allTables.append(table)

In [ ]:
def writeCsvTableToDisk(filename, table):
    with open(filename+'.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(table)
    return

In [ ]:
for t in range(len(allTables)):
    writeCsvTableToDisk('HeuristicV2 '+ str(t), allTables[t])

In [ ]:
# print(allTables[1])

In [ ]:
# %store data